## Bibliotecas e módulos

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
import pandas as pd
import csv

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn import svm

from matplotlib import rcParams
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

## Funções utilizadas

In [ ]:
#Função de leitura dos dados------------------------------------
def read_class_data(path):
    with open(path, newline='') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
            try:
                data = np.vstack( (data , np.asarray(row).astype(np.float) ) )
            except:
                data = np.asarray(row).astype(np.float)
    f.close()
    y = data[:,0]
    x = data[:,1:]
    return y,x

In [ ]:
#----------------------------
def compute_info(Z,x,posY):
    m,_ = x.shape
    
    mu = np.mean(x[:,Z],axis=0).reshape((1,len(Z))) #Média geral
    
    _x = x[:,Z]; _x = _x[posY[0],:]
    mu_j = np.mean(_x,axis=0).reshape((1,len(Z)))
    sig_j = np.cov(_x.T).reshape((len(Z),len(Z)))
    
    Sb = (len(posY[0])/m) * np.dot( (mu - mu_j).T,(mu - mu_j) )
    Sw = (len(posY[0])/m) * sig_j
    
    for j in range(1,len(labY)):
        _x = x[posY[j],:]; _x = _x[:,Z]
        mu_j = np.mean(_x,axis=0).reshape((1,len(Z)))
        sig_j = np.cov(_x.T).reshape((len(Z),len(Z)))
        Sb += (len(posY[0])/m) * np.dot( (mu - mu_j).T,(mu - mu_j) )
        Sw += (len(posY[0])/m) * sig_j

    return np.trace(Sb)/np.trace(Sw)


#----------------------------
def compute_info_gain_add(Z,W,x,posY):
    
    m,_ = x.shape

    vecJ = np.zeros(len(W))
    for at in range(len(W)):

        S = np.union1d(Z,W[at])
        S = np.int64(S).tolist()
        _x = x[:,S]; _x = _x[posY[0],:]

        mu = np.mean(x[:,S],axis=0).reshape((1,len(S))) #Média geral

        mu_j = np.mean(_x,axis=0).reshape((1,len(S)))
        sig_j = np.cov(_x.T).reshape((len(S),len(S)))
        #if len(S) ==1: sig_j = np.cov(_x.T) else: sig_j = np.cov(_x.T)

        Sb = (len(posY[0])/m) * np.dot( (mu - mu_j).T,(mu - mu_j) )
        Sw = (len(posY[0])/m) * sig_j

        for j in range(1,len(labY)):
            _x = x[posY[j],:]; _x = _x[:,S]
            mu_j = np.mean(_x,axis=0).reshape((1,len(S)))
            sig_j = np.cov(_x.T).reshape((len(S),len(S)))

            Sb += (len(posY[0])/m) * np.dot( (mu - mu_j).T,(mu - mu_j) )
            Sw += (len(posY[0])/m) * sig_j

        vecJ[at] = np.trace(Sb)/np.trace(Sw)
    
    return vecJ


#----------------------------
def compute_info_gain_remove(Z,x,posY):
    
    m,_ = x.shape

    vecJ = np.zeros(len(Z))
    for item in range(len(Z)):
        at = Z[item]

        S = Z.copy(); S.remove(at)
        _x = x[:,S]; _x = _x[posY[0],:]

        mu = np.mean(x[:,S],axis=0).reshape((1,len(S))) #Média geral

        mu_j = np.mean(_x,axis=0).reshape((1,len(S)))
        sig_j = np.cov(_x.T).reshape((len(S),len(S)))

        Sb = (len(posY[0])/m) * np.dot( (mu - mu_j).T,(mu - mu_j) )
        Sw = (len(posY[0])/m) * sig_j

        for j in range(1,len(labY)):
            _x = x[posY[j],:]; _x = _x[:,S]
            mu_j = np.mean(_x,axis=0).reshape((1,len(S)))
            sig_j = np.cov(_x.T).reshape((len(S),len(S)))

            Sb += (len(posY[0])/m) * np.dot( (mu - mu_j).T,(mu - mu_j) )
            Sw += (len(posY[0])/m) * sig_j

        vecJ[item] = np.trace(Sb)/np.trace(Sw)
    
    return vecJ

## Leitura dos dados - "Terremoto"
* Atributos são:
    * latitude
    * longitude
    * depth
    * mag
    * gap
    * dmin
    * rms
    * horizontalError
    * depthError
* Classes:
    * 1 - MB; (azul escuro)
    * 2 - MD; (ciano)
    * 3 - ML; (verde)
    * 4 - MW; (laranja)

In [ ]:
#Dados
path = 'EartQuakes_USGS_mar-abr2020.csv'

#Leitura dos dados
y,x = read_class_data(path)
y -= 1 #ajustar a classe para iniciar em 1

xOrig = np.copy(x)
x = preprocessing.normalize(xOrig, axis=0, norm='max')

## Processo de seleção de atributos

In [ ]:
#Numero de atributos a ser selecionado
atributos = [0,1,2,3,4,5,6,7,8]
n_sel = 4

#Conjuntos de atributos: a ideia é passar de W para Z
W = [i for i in atributos]
Z = []

#determina a posição de cada exemplo rotulado em x
labY ={}
for i in np.unique(y):
    pos = np.where(y == i)
    labY[i] = (pos[0]).tolist()

while len(Z) < n_sel:
    
    indAdd = np.argmax( compute_info_gain_add(Z,W,x,labY) )
    Z = np.int64(np.union1d( Z,W[indAdd] )).tolist()
    W.remove(W[indAdd])

    Jadd = compute_info(Z,x,labY)

    while True:
        if len(Z) <= 1: break
        indRem = np.argmin( compute_info_gain_remove(Z,x,labY) )
    
        tempZ = Z.copy(); tempZ.remove(Z[indRem])
        Jrem = compute_info(tempZ,x,labY)

        if (Jrem < Jadd) or (Z[indAdd] == Z[indRem]): break
        else: 
            indAdd = -1 #Garante comparação válida só no 1o loop
            W.append(Z[indRem])
            Z.remove(Z[indRem])

print('Atributos selecionados: ',Z)
selAtributos = np.int64(Z) #compatibilização do tipo

## Divisão treino/teste

In [ ]:
percentAvalia = 0.33
xD, xI, yD, yI = train_test_split(x,y,test_size=percentAvalia)

## Classificação com/sem seleção de atributos

In [ ]:
xD_sub = xD[:,selAtributos]
xI_sub = xI[:,selAtributos]

#Instanciação dos classificadores
svmLin = svm.SVC(C=100, kernel='linear',decision_function_shape='ovr')

#Treinamento e avaliação SEM atributos selecionados
svmLin.fit(xD,yD)
yEst = svmLin.predict(xI)
kappa_sem = cohen_kappa_score(yI, yEst)

#Treinamento e avaliação COM atributos selecionados
svmLin.fit(xD_sub,yD)
yEst = svmLin.predict(xI_sub)
kappa_com = cohen_kappa_score(yI, yEst)

print('Kappa sem seleção de atributos: ',kappa_sem)
print('Kappa com seleção de atributos: ',kappa_com)